# What Makes a Good Nigerian Movie?

In [12]:
import requests
import re
import numpy as np
import urllib.request
import pandas as pd
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from bs4 import BeautifulSoup as bs

In [48]:
url ="https://www.imdb.com/search/title/?country_of_origin=NG"
page = requests.get(url)
page

<Response [200]>

In [49]:
page_text = requests.get(url).text
doc = bs(page_text, "html.parser")
movies= doc.find(class_ = "lister list detail sub-list")
movie_containers = movies.find_all(class_= "lister-item mode-advanced")
movie_names=[]
for container in movie_containers:
    title= container.h3.a.text
    movie_names.append(title)
print(movie_names)

['Shanty Town', 'Blood Sisters', 'Brotherhood', 'Far from Home', 'Hey You!', 'Battle on Buka Street', 'The Wait', 'After Party', 'Passport', 'Amina', 'Half of a Yellow Sun', 'Star Girl', 'In Line', 'God Calling', 'Ijakumo: The Born Again Stripper', 'King of Boys: The Return of the King', 'Lionheart', 'Beautiful', "Elesin Oba: The King's Horseman", 'Elevator Baby', 'The Best of Us', 'A Sunday Affair', 'The Set Up', 'Last Flight to Abuja', 'All Things Fair', 'Love, Sex, Religion', 'Mother of George', 'Leaked', 'Man of God', 'Daughters', 'Òlòturé', 'Blinded', 'The Griot', 'Living in Bondage: Breaking Free', 'Namaste Wahala', 'Benediction', 'Things Fall Apart', 'Halita', 'Anikulapo', 'Citation', 'Coming from Insanity', 'The Wedding Party', 'True Lies 2', 'The Smart Money Woman', 'Small Chops', 'Unmarried - Nigeria', 'Black November', 'The Set Up 2', 'Omo Ghetto: The Saga', 'Nneka the Pretty Serpent']


### Web Scraping Function

In [42]:
def get_movie_data(url):
    page_text = requests.get(url).text
    doc = bs(page_text, "html.parser")
    movies= doc.find(class_ = "lister list detail sub-list")
    movie_containers = movies.find_all(class_= "lister-item mode-advanced")
    movie_names=[]
    for container in movie_containers:
        title= container.h3.a.text
        movie_names.append(title)
    year_containers = movies.find_all(class_ = "lister-item-year text-muted unbold")
    movie_years=[]
    for year in year_containers:
        movie_years.append(year.text) 
    duration_containers = movies.find_all(class_ = "runtime")
    movie_duration=[]
    for duration in duration_containers:
        movie_duration.append(duration.text)
    genre_containers = movies.find_all(class_ = "genre")
    movie_genre=[]
    for genre in genre_containers:
        movie_genre.append(genre.text)
    rating_containers = movies.find_all(class_ = "inline-block ratings-imdb-rating")
    movie_rating=[]
    for rating in rating_containers:
        movie_rating.append(rating.text)
    star_containers = movies.find_all(class_ = "lister-item-content")
    movie_stars=[]
    for star in star_containers:
        p_links = star.find_all("a", id= False)
        movie_stars.append(p_links[12:])
    votes_containers = movies.find_all(class_ = "sort-num_votes-visible")
    movie_votes=[]
    for vote in votes_containers:
        movie_votes.append(vote.text)
    return pd.DataFrame(list(zip(movie_names, movie_years, movie_duration, movie_genre, movie_rating, movie_stars, movie_votes
                          )), columns=["Names","Years", "Duration", "Genre", "Rating", "Stars", "Votes"])

In [43]:
df1 = get_movie_data("https://www.imdb.com/search/title/?country_of_origin=NG")
df1.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,(2023– ),40 min,"\nAction, Crime, Drama",\n\n4.7\n,"[[Sola Sobowale], [Nancy Isime], [Richard Mofe...",\nVotes:\n277\n
1,Blood Sisters,(2022),224 min,"\nCrime, Drama, Thriller",\n\n5.5\n,"[[Nancy Isime], [Ini Dima Okojie], [Uche Jombo...","\nVotes:\n1,017\n"
2,Brotherhood,(I) (2022),120 min,"\nAction, Crime",\n\n7.8\n,"[[Loukman Ali], [Jide Kene Achufusi], [Adetayo...",\nVotes:\n99\n
3,Far from Home,(2022– ),32 min,\nDrama,\n\n5.8\n,"[[Richard Mofe-Damijo], [Mike Afolarin], [Ades...",\nVotes:\n292\n
4,Hey You!,(I) (2022),104 min,"\nRomance, Thriller",\n\n3.9\n,"[[Uyoyou Adia], [John Promise Adurable], [Tunb...",\nVotes:\n91\n


In [44]:
df2 = get_movie_data("")
df2.head()

MissingSchema: Invalid URL '': No schema supplied. Perhaps you meant http://?

In [ ]:
df3 = get_movie_data("")

In [ ]:
###Data Cleaning Function

In [ ]:
def clean_dataframe(df):
    def cleaner_g1_y1(text):
        return re.sub('[^A-Za-z0-9]', ' ',text)
    df['Genre'] = df['Genre'].apply(cleaner_g1_y1)
    df['Years'] = df['Years'].apply(cleaner_g1_y1)
    def cleaner_v1(text):
        return re.sub('[^A-Za-z0-9]', '',text)
        df['Votes'] = df['Votes'].apply(cleaner_v1)
    def cleaner_r1(text):
        return re.sub('\n', '',text)
    df['Rating'] = df['Rating'].apply(cleaner_r1)
    def cleaner_y2(text):
        return re.sub('[I, \s]', '',text)
    df['Years'] = df['Years'].apply(cleaner_y2)
    def cleaner_s1(text):
        text = str(text)[1:-1]             
        return text
    df['Stars'] = df['Stars'].apply(cleaner_s1)
    def cleaner_d1_v2_s2(text):
        return re.sub('[^0-9]', '',text)
    df['Duration'] = df['Duration'].apply(cleaner_d1_v2_s2)
    df['Votes'] = df['Votes'].apply(cleaner_d1_v2_s2)
    def cleaner_s3(text):
        text = re.sub('<.*?>','',text)            
        return text
    df['Stars'] = df['Stars'].apply(cleaner_s3)
    return df.head()

In [45]:
clean_dataframe(df1)

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,2023,40,Action Crime Drama,4.7,"Sola Sobowale, Nancy Isime, Richard Mofe-Damij...",277
1,Blood Sisters,2022,224,Crime Drama Thriller,5.5,"Nancy Isime, Ini Dima Okojie, Uche Jombo, Kehi...",1017
2,Brotherhood,2022,120,Action Crime,7.8,"Loukman Ali, Jide Kene Achufusi, Adetayo Adebo...",99
3,Far from Home,2022,32,Drama,5.8,"Richard Mofe-Damijo, Mike Afolarin, Adesua Eto...",292
4,Hey You!,2022,104,Romance Thriller,3.9,"Uyoyou Adia, John Promise Adurable, Tunbosun A...",91


In [47]:
df1

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,2023,40,Action Crime Drama,4.7,"Sola Sobowale, Nancy Isime, Richard Mofe-Damij...",277
1,Blood Sisters,2022,224,Crime Drama Thriller,5.5,"Nancy Isime, Ini Dima Okojie, Uche Jombo, Kehi...",1017
2,Brotherhood,2022,120,Action Crime,7.8,"Loukman Ali, Jide Kene Achufusi, Adetayo Adebo...",99
3,Far from Home,2022,32,Drama,5.8,"Richard Mofe-Damijo, Mike Afolarin, Adesua Eto...",292
4,Hey You!,2022,104,Romance Thriller,3.9,"Uyoyou Adia, John Promise Adurable, Tunbosun A...",91
5,Battle on Buka Street,2022,141,Comedy,8.7,"Funke Akindele, Tobi Makinde, Bimbo Ademoye, F...",42
6,The Wait,2021,122,Drama,6.2,"Fiyin Gambo, Yemi Morafa, Ope Ade-Agbo, Mike A...",22
7,After Party,2021,105,Comedy,7.5,"Tope Alake, Funny Bone, Timo Elliott, Peggy He...",17
8,Passport,2022,111,Comedy,8.1,"Add a Plot, Dimeji Ajibola, Daniel Abua, Latee...",13
9,Amina,2021,116,Action Adventure Biography,4.0,"Izu Ojukwu, Usman Tijani Abubakar, Jennifer Ez...",385


In [52]:
page_text = requests.get(url).text
doc = bs(page_text, "html.parser")
movies= doc.find(class_ = "lister list detail sub-list")
movie_containers = movies.find_all(class_= "lister-item mode-advanced")
movie_names=[]
for container in movie_containers:
    title= container.h3.a.text
    movie_names.append(title)
print(movie_names)
print(len(movie_names))

['Shanty Town', 'Blood Sisters', 'Brotherhood', 'Far from Home', 'Hey You!', 'Battle on Buka Street', 'The Wait', 'After Party', 'Passport', 'Amina', 'Half of a Yellow Sun', 'Star Girl', 'In Line', 'God Calling', 'Ijakumo: The Born Again Stripper', 'King of Boys: The Return of the King', 'Lionheart', 'Beautiful', "Elesin Oba: The King's Horseman", 'Elevator Baby', 'The Best of Us', 'A Sunday Affair', 'The Set Up', 'Last Flight to Abuja', 'All Things Fair', 'Love, Sex, Religion', 'Mother of George', 'Leaked', 'Man of God', 'Daughters', 'Òlòturé', 'Blinded', 'The Griot', 'Living in Bondage: Breaking Free', 'Namaste Wahala', 'Benediction', 'Things Fall Apart', 'Halita', 'Anikulapo', 'Citation', 'Coming from Insanity', 'The Wedding Party', 'True Lies 2', 'The Smart Money Woman', 'Small Chops', 'Unmarried - Nigeria', 'Black November', 'The Set Up 2', 'Omo Ghetto: The Saga', 'Nneka the Pretty Serpent']
50


In [53]:
year_containers = movies.find_all(class_ = "lister-item-year text-muted unbold")
movie_years=[]
for year in year_containers:
    movie_years.append(year.text)
print(len(movie_years))

50


In [54]:
duration_containers = movies.find_all(class_ = "runtime")
movie_duration=[]
for duration in duration_containers:
    movie_duration.append(duration.text)
print(len(movie_duration))

41


In [55]:
genre_containers = movies.find_all(class_ = "genre")
movie_genre=[]
for genre in genre_containers:
    movie_genre.append(genre.text)
print(len(movie_genre))

50


In [56]:
rating_containers = movies.find_all(class_ = "inline-block ratings-imdb-rating")
movie_rating=[]
for rating in rating_containers:
    movie_rating.append(rating.text)
print(len(movie_rating))

40


In [80]:
star_containers = movies.find_all(class_ = "lister-item-content")
movie_stars=[]
for star in star_containers:
    p_links = star.find_all("a", id= False)
    movie_stars.append(p_links[12:])
movie_stars

[[<a href="/name/nm1417032/">Sola Sobowale</a>,
  <a href="/name/nm7484947/">Nancy Isime</a>,
  <a href="/name/nm1816193/">Richard Mofe-Damijo</a>,
  <a href="/name/nm1421120/">Chidi Mokeme</a>],
 [<a href="/name/nm7484947/">Nancy Isime</a>,
  <a href="/name/nm8497258/">Ini Dima Okojie</a>,
  <a href="/name/nm2237094/">Uche Jombo</a>,
  <a href="/name/nm3535138/">Kehinde Bankole</a>],
 [<a href="/name/nm8383036/">Loukman Ali</a>,
  <a href="/name/nm7975075/">Jide Kene Achufusi</a>,
  <a href="/name/nm14508813/">Adetayo Adebowale</a>,
  <a href="/name/nm12354481/">Adebowale Adedayo</a>,
  <a href="/name/nm14508795/">Kehinde Adeyanu</a>],
 [<a href="/name/nm1816193/">Richard Mofe-Damijo</a>,
  <a href="/name/nm10404026/">Mike Afolarin</a>,
  <a href="/name/nm6767239/">Adesua Etomi-Wellington</a>,
  <a href="/name/nm2481000/">Funke Akindele</a>],
 [<a href="/name/nm10734389/">Uyoyou Adia</a>,
  <a href="/name/nm14086798/">John Promise Adurable</a>,
  <a href="/name/nm4931261/">Tunbosun Ai

In [58]:
votes_containers = movies.find_all(class_ = "sort-num_votes-visible")
movie_votes=[]
for vote in votes_containers:
    movie_votes.append(vote.text)
print(movie_votes)

['\nVotes:\n276\n', '\nVotes:\n1,017\n', '\nVotes:\n93\n', '\nVotes:\n292\n', '\nVotes:\n91\n', '\nVotes:\n42\n', '\nVotes:\n20\n', '\nVotes:\n17\n', '\nVotes:\n13\n', '\nVotes:\n384\n', '\nVotes:\n2,045\n| Gross:\n$0.05M\n', '\nVotes:\n44\n', '\nVotes:\n157\n', '\nVotes:\n12\n', '\nVotes:\n282\n', '\nVotes:\n1,503\n', '\nVotes:\n397\n', '\nVotes:\n103\n', '\nVotes:\n149\n', '\nVotes:\n369\n', '\nVotes:\n1,223\n| Gross:\n$0.15M\n', '\nVotes:\n171\n', '\nVotes:\n22\n', '\nVotes:\n830\n', '\nVotes:\n28\n', '\nVotes:\n142\n', '\nVotes:\n1,587\n', '\nVotes:\n50\n', '\nVotes:\n30\n', '\nVotes:\n376\n', '\nVotes:\n1,000\n', '\nVotes:\n127\n', '\nVotes:\n641\n', '\nVotes:\n12\n', '\nVotes:\n121\n', '\nVotes:\n34\n', '\nVotes:\n664\n', '\nVotes:\n43\n', '\nVotes:\n77\n', '\nVotes:\n141\n']


In [314]:
df = pd.DataFrame(list(zip(movie_names, movie_years, movie_duration, movie_genre, movie_rating, movie_stars, movie_votes
                          )), columns=["Names","Years", "Duration", "Genre", "Rating", "Stars", "Votes"])
print(df)

                                   Names         Years Duration  \
0                            Shanty Town      (2023– )   40 min   
1                          Blood Sisters        (2022)  224 min   
2                            Brotherhood    (I) (2022)  120 min   
3                          Far from Home      (2022– )   32 min   
4                               Hey You!    (I) (2022)  104 min   
5                  Battle on Buka Street        (2022)  141 min   
6                               The Wait  (III) (2021)  122 min   
7                            After Party        (2021)  105 min   
8                               Passport        (2022)  111 min   
9                                  Amina        (2021)  116 min   
10                  Half of a Yellow Sun        (2013)  115 min   
11                             Star Girl        (2021)  120 min   
12                               In Line        (2017)  117 min   
13                           God Calling        (2018)  439 mi

In [315]:
df.to_csv("NigerianMovies.csv", index= False)

### Data Cleaning

In [316]:
df.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,(2023– ),40 min,"\nAction, Crime, Drama",\n\n4.7\n,"[[Sola Sobowale], [Nancy Isime], [Richard Mofe...",\nVotes:\n276\n
1,Blood Sisters,(2022),224 min,"\nCrime, Drama, Thriller",\n\n5.5\n,"[[Nancy Isime], [Ini Dima Okojie], [Uche Jombo...","\nVotes:\n1,017\n"
2,Brotherhood,(I) (2022),120 min,"\nAction, Crime",\n\n8.0\n,"[[Loukman Ali], [Jide Kene Achufusi], [Adetayo...",\nVotes:\n93\n
3,Far from Home,(2022– ),32 min,\nDrama,\n\n5.8\n,"[[Richard Mofe-Damijo], [Mike Afolarin], [Ades...",\nVotes:\n292\n
4,Hey You!,(I) (2022),104 min,"\nRomance, Thriller",\n\n3.9\n,"[[Uyoyou Adia], [John Promise Adurable], [Tunb...",\nVotes:\n91\n


In [317]:
df.isnull().sum()

Names       0
Years       0
Duration    0
Genre       0
Rating      0
Stars       0
Votes       0
dtype: int64

In [318]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Names     40 non-null     object
 1   Years     40 non-null     object
 2   Duration  40 non-null     object
 3   Genre     40 non-null     object
 4   Rating    40 non-null     object
 5   Stars     40 non-null     object
 6   Votes     40 non-null     object
dtypes: object(7)
memory usage: 2.3+ KB


In [319]:
df.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,(2023– ),40 min,"\nAction, Crime, Drama",\n\n4.7\n,"[[Sola Sobowale], [Nancy Isime], [Richard Mofe...",\nVotes:\n276\n
1,Blood Sisters,(2022),224 min,"\nCrime, Drama, Thriller",\n\n5.5\n,"[[Nancy Isime], [Ini Dima Okojie], [Uche Jombo...","\nVotes:\n1,017\n"
2,Brotherhood,(I) (2022),120 min,"\nAction, Crime",\n\n8.0\n,"[[Loukman Ali], [Jide Kene Achufusi], [Adetayo...",\nVotes:\n93\n
3,Far from Home,(2022– ),32 min,\nDrama,\n\n5.8\n,"[[Richard Mofe-Damijo], [Mike Afolarin], [Ades...",\nVotes:\n292\n
4,Hey You!,(I) (2022),104 min,"\nRomance, Thriller",\n\n3.9\n,"[[Uyoyou Adia], [John Promise Adurable], [Tunb...",\nVotes:\n91\n


In [320]:
def cleaner(text):
    return re.sub('[^A-Za-z0-9]', ' ',text)
df['Genre'] = df['Genre'].apply(cleaner)
df['Years'] = df['Years'].apply(cleaner)
#df['Rating'] = df['Rating'].apply(cleaner)
#df['Votes'] = df['Votes'].apply(cleaner)
#df['Stars'] = df['Stars'].apply(cleaner)

In [321]:
df.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,2023,40 min,Action Crime Drama,\n\n4.7\n,"[[Sola Sobowale], [Nancy Isime], [Richard Mofe...",\nVotes:\n276\n
1,Blood Sisters,2022,224 min,Crime Drama Thriller,\n\n5.5\n,"[[Nancy Isime], [Ini Dima Okojie], [Uche Jombo...","\nVotes:\n1,017\n"
2,Brotherhood,I 2022,120 min,Action Crime,\n\n8.0\n,"[[Loukman Ali], [Jide Kene Achufusi], [Adetayo...",\nVotes:\n93\n
3,Far from Home,2022,32 min,Drama,\n\n5.8\n,"[[Richard Mofe-Damijo], [Mike Afolarin], [Ades...",\nVotes:\n292\n
4,Hey You!,I 2022,104 min,Romance Thriller,\n\n3.9\n,"[[Uyoyou Adia], [John Promise Adurable], [Tunb...",\nVotes:\n91\n


In [322]:
def cleaner(text):
    return re.sub('[^A-Za-z0-9]', '',text)
df['Votes'] = df['Votes'].apply(cleaner)
df.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,2023,40 min,Action Crime Drama,\n\n4.7\n,"[[Sola Sobowale], [Nancy Isime], [Richard Mofe...",Votes276
1,Blood Sisters,2022,224 min,Crime Drama Thriller,\n\n5.5\n,"[[Nancy Isime], [Ini Dima Okojie], [Uche Jombo...",Votes1017
2,Brotherhood,I 2022,120 min,Action Crime,\n\n8.0\n,"[[Loukman Ali], [Jide Kene Achufusi], [Adetayo...",Votes93
3,Far from Home,2022,32 min,Drama,\n\n5.8\n,"[[Richard Mofe-Damijo], [Mike Afolarin], [Ades...",Votes292
4,Hey You!,I 2022,104 min,Romance Thriller,\n\n3.9\n,"[[Uyoyou Adia], [John Promise Adurable], [Tunb...",Votes91


In [323]:
def cleaner(text):
    return re.sub('\n', '',text)
df['Rating'] = df['Rating'].apply(cleaner)
df.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,2023,40 min,Action Crime Drama,4.7,"[[Sola Sobowale], [Nancy Isime], [Richard Mofe...",Votes276
1,Blood Sisters,2022,224 min,Crime Drama Thriller,5.5,"[[Nancy Isime], [Ini Dima Okojie], [Uche Jombo...",Votes1017
2,Brotherhood,I 2022,120 min,Action Crime,8.0,"[[Loukman Ali], [Jide Kene Achufusi], [Adetayo...",Votes93
3,Far from Home,2022,32 min,Drama,5.8,"[[Richard Mofe-Damijo], [Mike Afolarin], [Ades...",Votes292
4,Hey You!,I 2022,104 min,Romance Thriller,3.9,"[[Uyoyou Adia], [John Promise Adurable], [Tunb...",Votes91


In [324]:
def cleaner(text):
    return re.sub('[I, \s]', '',text)
df['Years'] = df['Years'].apply(cleaner)
df.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,2023,40 min,Action Crime Drama,4.7,"[[Sola Sobowale], [Nancy Isime], [Richard Mofe...",Votes276
1,Blood Sisters,2022,224 min,Crime Drama Thriller,5.5,"[[Nancy Isime], [Ini Dima Okojie], [Uche Jombo...",Votes1017
2,Brotherhood,2022,120 min,Action Crime,8.0,"[[Loukman Ali], [Jide Kene Achufusi], [Adetayo...",Votes93
3,Far from Home,2022,32 min,Drama,5.8,"[[Richard Mofe-Damijo], [Mike Afolarin], [Ades...",Votes292
4,Hey You!,2022,104 min,Romance Thriller,3.9,"[[Uyoyou Adia], [John Promise Adurable], [Tunb...",Votes91


In [325]:
def cleaner(text):
    return re.sub('[^0-9]', '',text)
df['Duration'] = df['Duration'].apply(cleaner)
df['Votes'] = df['Votes'].apply(cleaner)

In [326]:
df.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,2023,40,Action Crime Drama,4.7,"[[Sola Sobowale], [Nancy Isime], [Richard Mofe...",276
1,Blood Sisters,2022,224,Crime Drama Thriller,5.5,"[[Nancy Isime], [Ini Dima Okojie], [Uche Jombo...",1017
2,Brotherhood,2022,120,Action Crime,8.0,"[[Loukman Ali], [Jide Kene Achufusi], [Adetayo...",93
3,Far from Home,2022,32,Drama,5.8,"[[Richard Mofe-Damijo], [Mike Afolarin], [Ades...",292
4,Hey You!,2022,104,Romance Thriller,3.9,"[[Uyoyou Adia], [John Promise Adurable], [Tunb...",91


In [327]:
def cleaner(text):
    text = str(text)[1:-1]             
    return text
df['Stars'] = df['Stars'].apply(cleaner)
df.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,2023,40,Action Crime Drama,4.7,"<a href=""/name/nm1417032/"">Sola Sobowale</a>, ...",276
1,Blood Sisters,2022,224,Crime Drama Thriller,5.5,"<a href=""/name/nm7484947/"">Nancy Isime</a>, <a...",1017
2,Brotherhood,2022,120,Action Crime,8.0,"<a href=""/name/nm8383036/"">Loukman Ali</a>, <a...",93
3,Far from Home,2022,32,Drama,5.8,"<a href=""/name/nm1816193/"">Richard Mofe-Damijo...",292
4,Hey You!,2022,104,Romance Thriller,3.9,"<a href=""/name/nm10734389/"">Uyoyou Adia</a>, <...",91


In [329]:
def cleaner(text):
    text = re.sub('<.*?>','',text)            
    return text
df['Stars'] = df['Stars'].apply(cleaner)
df.head()

,Names,Years,Duration,Genre,Rating,Stars,Votes
0,Shanty Town,2023,40,Action Crime Drama,4.7,"Sola Sobowale, Nancy Isime, Richard Mofe-Damij...",276
1,Blood Sisters,2022,224,Crime Drama Thriller,5.5,"Nancy Isime, Ini Dima Okojie, Uche Jombo, Kehi...",1017
2,Brotherhood,2022,120,Action Crime,8.0,"Loukman Ali, Jide Kene Achufusi, Adetayo Adebo...",93
3,Far from Home,2022,32,Drama,5.8,"Richard Mofe-Damijo, Mike Afolarin, Adesua Eto...",292
4,Hey You!,2022,104,Romance Thriller,3.9,"Uyoyou Adia, John Promise Adurable, Tunbosun A...",91
